# ML - Problem Set 2: Predicting Poverty

El objetivo principal es construir un modelo predictivo de la pobreza de los hogares. Nótese que un hogar se clasifica como

\begin{equation}
    Poor = I \left( Inc < Pl \right)
\end{equation}

donde $I$ es una función indicadora que toma uno si el ingreso familiar está por debajo de una determinada línea de pobreza.

In [ ]:
# Importar librerias
import pandas as pd
import numpy as np

In [ ]:
# Importar datos

df_1 = pd.read_csv('sample_submission.csv')
df_2 = pd.read_csv('test_hogares.csv')
df_3 = pd.read_csv('test_personas.csv')
df_4 = pd.read_csv('train_hogares.csv')
df_5 = pd.read_csv('train_personas.csv')

In [19]:
df_1

,id,pobre
0,2a7ddc2779480d7f19834953,0
1,a0c2e751e582fd49d564f308,0
2,57273d19e8464a5ff66a582b,1
3,418d052ff7878940ab938601,0
4,212a37fc17016a3c78f76852,1
...,...,...
66163,e7d3c7cc62acd33867b7b7ee,0
66164,710db9f02de9c616f846c72e,0
66165,5a0d6b6fad06e4736fb6d95f,0
66166,569496964b70185785b57789,1
